In [1]:
import pickle
from keras.optimizers import Adam
from keras.models import Sequential, Model
from keras.layers import Dense, Embedding, LSTM, Add, Input, Dropout, Bidirectional
%run clean_text.ipynb

Using TensorFlow backend.


Number of unique words in the vocabulary : 1792
Maximum length of caption is :  39


In [2]:
with open('embeddings', 'rb') as fp :
    embeddings = pickle.load(fp)

In [3]:
model_i = Input(shape = (2048,))
model_dr1 = Dropout(0.5)(model_i)
model_d = Dense(512)(model_dr1)

In [4]:
model_t = Input(shape = (39,))
model_e = Embedding(1792, 200)(model_t)
model_dr2 = Dropout(0.5)(model_e)
model_l = Bidirectional(LSTM(256))(model_dr2)

In [5]:
model_c = Add()([model_d, model_l])
model_cd1 = Dense(512)(model_c)
model_cd2 = Dense(1792, activation = 'softmax')(model_cd1)

In [6]:
model = Model(inputs = [model_i, model_t], output = model_cd2)

C:\Anaconda\envs\deeplearning\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`
  """Entry point for launching an IPython kernel.


In [7]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 39)           0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, 2048)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 39, 200)      358400      input_2[0][0]                    
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 2048)         0           input_1[0][0]                    
____________________________________________________________________________________________

In [8]:
model.layers[2].set_weights([embeddings])
model.layers[2].trainable = False
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [9]:
generator =  data_generator(text_dict, word_index, 10, transferred_features, maxlen, num_classes)
model.fit_generator(generator, epochs = 1, verbose = 1, steps_per_epoch = 700)
model.save('model_1.h5')

Epoch 1/1
700/700 [==============================] - 354s 506ms/step - loss: 3.9960


In [14]:
model.fit_generator(generator, epochs = 1, verbose = 1, steps_per_epoch = 700)
model.save('model_1.h5')

Epoch 1/1
700/700 [==============================] - 353s 505ms/step - loss: 3.3595


In [18]:
model.fit_generator(generator, epochs = 1, verbose = 1, steps_per_epoch = 700)
model.save('model_1.h5')

Epoch 1/1
700/700 [==============================] - 356s 509ms/step - loss: 3.1868


In [19]:
def generateCaption(photo):
    in_text = 'start'
    photo = np.reshape(photo, (1,2048))
    for i in range(maxlen):
        sequences = [word_index[w] for w in in_text.split() if w in word_index]
        sequences = sequence.pad_sequences([sequences], maxlen=maxlen)
        yhat = model.predict([photo, sequences], verbose=0)
        yhat = np.argmax(yhat)
        word = index_word[yhat]
        in_text += ' ' + word
        if word == 'end':
            break
    final = in_text.split()
    final = final[1:-1]
    final = ' '.join(final)
    return final

In [20]:
with open('transfer_train_data', 'rb') as fp :
    images = pickle.load(fp)

In [28]:
generateCaption(images['3681414069_71ba164f71'])

'a dog is laying a smaller of a dog'